# Analytic order of Sha

Exercise 1. Write a function which takes as input an elliptic curve over Q, and outputs the analytic order of sha.

In [24]:
#preliminaryfunctions:
#compute Omega
def period(E):
    factor =  1 + 1/2*(1+sgn(E.discriminant()))
    b2 = E.b2()
    b4 = E.b4()
    b6 = E.b6()
    f = 4*x^3 + b2*x^2+2*b4*x + b6
    print(f)
    roots = f.roots(ring=CC)
    real_roots = [p[0] for p in roots if p[0].is_real()]
    nonreal_roots = [p[0] for p in roots if not p[0].is_real()]
    if len(real_roots) == 3:
        real_roots.sort(reverse=true)
        difference = [real_roots[0]-real_roots[1],real_roots[0]-real_roots[2]]
        a0 = sqrt(CC(difference[0]))
        b0 = sqrt(CC(difference[1]))
        if abs(a0-b0)>abs(a0+b0):
                     b0 =-b0
        P = a0.agm(b0)
        return(CC(factor*pi/P))
    else:
        e3 = real_roots[0]
        diff = e3-nonreal_roots[0]
        z = sqrt(diff)
        if z.real()<0:
            z = -z
        P = z.agm(z.conjugate())    
   # e3 = roots[2][0]
   # e1 = roots[0][0]
   # e2 = roots[1][0]
   # ees1 = [e1,e2,e3]
   # ees2 = [e2,e1,e3]
   # ees3 = [e3,e1,e2]
   # listees = [ees1,ees2,ees3]
    #ees.sort(reverse = true)
    #print(ees)
   # periods = []
   # for ees in listees:
   #     difference = [ees[0]-ees[1],ees[0]-ees[2]]
   #     a0 = sqrt(CC(difference[0]))
   #     b0 = sqrt(CC(difference[1]))
   #     if abs(a0-b0)>abs(a0+b0):
    #                 b0 =-b0
    #    periods.append(a0.agm(b0))
   # factor =  1 + 1/2*(1+sgn(E.discriminant()))
 #   print(periods)
   # P = 1
   # for Ps in periods:
     #   if imag(Ps)==0:
     #       P = Ps
    return(CC(factor*pi/P))
E = EllipticCurve([0, 0, 1, -7, 36])
period(E)
def regul(E):
    h = E.height_function()
    def pairing(x,y):
        z = 1/2*(h(x+y)-h(x)-h(y))
        return z
    genos = E.gens()
    mat = matrix(CC, len(genos), lambda i , j : pairing(genos[i],genos[j]))
    return(mat.det())


4*x^3 - 28*x + 145


In [25]:
def analytic_sha_order(E):
    leading_coeff = CC(sympow.analytic_rank(E)[1])
    torsion_cardinality = CC(E.torsion_subgroup().cardinality())
    omega = period(E)
    tamagawa_product = E.tamagawa_product()
    regulator = regul(E)
    print('Coefficient: '+str(leading_coeff))
    print('Torsion_Cardinality: '+str(torsion_cardinality))
    print('Real Period: '+str(omega))
    print('TamagawaProduct: '+str(tamagawa_product))
    print('Regulator: '+str(regul(E)))
    Shan = (torsion_cardinality*torsion_cardinality *leading_coeff) / (omega*regulator*tamagawa_product) 
    return(Shan)
    # TODO
analytic_sha_order(E)

4*x^3 - 28*x + 145
Coefficient: 8.17379000000000
Torsion_Cardinality: 1.00000000000000
Real Period: 2.47212704804395
TamagawaProduct: 1
Regulator: 3.30637847459134


1.00000028651251

Exercise 2. Run your function on several of the elliptic curves in the LMFDB using the API, and verify your result with the order given there.

Exercise 3. Can you extend the function to arbitrary number fields?